In [1]:
from numpy import *
from numpy.random import *
from TrianglePlot import *
import pandas

df = pandas.read_csv('../GAIA_SDSS_substructures_next_30_cands.csv')
names = df.group_id.unique()
num_cands = size(names)

for i_cand in range(0,1):
    Cand = df.loc[df['group_id'] == names[i_cand]]
    name = names[i_cand]
    nstars = size(Cand,0)
    vx = Cand.GalRVel
    vy = Cand.GalTVel
    vz = Cand.GalzVel
    feh = Cand.feh
    VelocityTriangle(Cand)
    print i_cand

0


In [5]:
# S1
df = pandas.read_csv('../S1_only_cat_simple.csv')
name = 'S1'
vx = df.GalRVel
vy = df.GalphiVel
vz = df.GalzVel
feh = df.feh
fig = VelocityTriangle(vx,vy,vz,feh,name)


In [2]:
Cand.group_id

0     NCand0
1     NCand0
2     NCand0
3     NCand0
4     NCand0
5     NCand0
6     NCand0
7     NCand0
8     NCand0
9     NCand0
10    NCand0
11    NCand0
12    NCand0
13    NCand0
14    NCand0
15    NCand0
16    NCand0
17    NCand0
18    NCand0
19    NCand0
20    NCand0
21    NCand0
22    NCand0
23    NCand0
24    NCand0
25    NCand0
26    NCand0
27    NCand0
28    NCand0
Name: group_id, dtype: object